In [107]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['df_2000_2001.csv.gz',
 'df_2000_2001_budget_revenue_title_cert.csv.gz',
 'df_2002-2014_no_2007(2).csv.gz',
 'df_2002-2014_no_2007.csv.gz',
 'df_clean_basics.csv.gz',
 'final_basics_df.csv.gz',
 'final_genres_df.csv.gz',
 'final_ratings_df.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2002.csv.gz',
 'final_tmdb_data_2003.csv.gz',
 'final_tmdb_data_2004.csv.gz',
 'final_tmdb_data_2005.csv.gz',
 'final_tmdb_data_2006.csv.gz',
 'final_tmdb_data_2008.csv.gz',
 'final_tmdb_data_2009.csv.gz',
 'final_tmdb_data_2010.csv.gz',
 'final_tmdb_data_2011.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'final_tmdb_data_2013.csv.gz',
 'final_tmdb_data_2014.csv.gz',
 'final_tmdb_df.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2002.json',
 'tmdb_api_results_2003.json',
 'tmdb_api_results_2004.json',
 'tmdb_api_results_2005.json',
 'tmdb_api_results_2

In [108]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

In [109]:
basics = pd.read_csv(basics_url,sep='\t', low_memory=False)

KeyboardInterrupt: 

In [ ]:
basics.head()

In [ ]:
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [ ]:
ratings = pd.read_csv(ratings_url,sep='\t', low_memory=False)

In [ ]:
ratings.head()

In [ ]:
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

In [ ]:
akas = pd.read_csv(akas_url,sep='\t', low_memory=False)

In [ ]:
akas.head()

In [ ]:
basics = basics.replace({'\\N':np.nan})

In [ ]:
akas = akas.replace({'\\N':np.nan})

In [ ]:
ratings = ratings.replace({'\\N':np.nan})

In [ ]:
basics.info()

In [ ]:
basics['runtimeMinutes'].value_counts()

In [ ]:
basics.duplicated().sum()

In [ ]:
basics.isna().sum()

In [ ]:
basics.dropna(subset=['runtimeMinutes'], inplace=True)
basics.isna().sum()

In [ ]:
basics.dropna(subset=['genres'], inplace=True)
basics.isna().sum()

In [ ]:
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]
basics.isna().sum()

In [ ]:
basics['titleType'].value_counts()

In [ ]:
basics = basics[basics["titleType"].str.contains("movie")==True]
basics['titleType'].value_counts()

In [ ]:
basics['startYear'].value_counts()

In [ ]:
basics.dropna(subset=['startYear'], inplace=True)
basics.isna().sum()

In [ ]:
basics = basics[basics['startYear'] >= '2000']
basics['startYear'].value_counts()

In [ ]:
basics = basics[basics['startYear'] <= '2021']
basics['startYear'].value_counts()

In [ ]:
keepers =basics['tconst'].isin(akas['titleId'])
keepers

In [ ]:
basics = basics[keepers]
basics

In [ ]:
akas.head()

In [ ]:
akas['region'].value_counts()

In [ ]:
akas = akas[akas["region"].str.contains("US")==True]
akas['region'].value_counts()

In [ ]:
akas.info()

In [ ]:
basics.info()

In [ ]:
ratings.info()

In [ ]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

In [ ]:
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [ ]:
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [ ]:
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [ ]:
import json
with open('C:/Users/fid24/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

In [ ]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key-tmdb']

In [119]:
YEARS_TO_GET = [2007] #2002,2003,2004,2005,2006,2008,2009,2010,2011,2012,2013,2014, i was able to get all the years i wante except 2007.

In [120]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

In [121]:
 def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [122]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    # If it does not exist: create it
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:    
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    # Load in the dataframe from project part 1 as basics:
    basics = pd.read_csv('Data/title_basics.csv.gz')
    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: Expected object or value

In [ ]:
with open('C:/Users/fid24/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

In [ ]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key-tmdb']

In [ ]:
JSON_FILE_2002 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2002.csv.gz"
JSON_FILE_2002

In [ ]:
df_2002 = pd.read_csv(JSON_FILE_2002)
df_2002.head()

In [ ]:
JSON_FILE_2003 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2003.csv.gz"
JSON_FILE_2003

In [ ]:
df_2003 = pd.read_csv(JSON_FILE_2003)
df_2003.head()

In [ ]:
JSON_FILE_2004 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2004.csv.gz"
JSON_FILE_2004

In [ ]:
df_2004 = pd.read_csv(JSON_FILE_2004)
df_2004.head()

In [ ]:
JSON_FILE_2005 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2005.csv.gz"
JSON_FILE_2005

In [ ]:
df_2005 = pd.read_csv(JSON_FILE_2005)
df_2005.head()

In [ ]:
JSON_FILE_2006 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2006.csv.gz"
JSON_FILE_2006

In [ ]:
df_2006 = pd.read_csv(JSON_FILE_2006)
df_2006.head()

In [ ]:
JSON_FILE_2007 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2007.csv.gz"
JSON_FILE_2007

In [ ]:
df_2007 = pd.read_csv(JSON_FILE_2007)
df_2007.head()

In [ ]:
JSON_FILE_2008 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2008.csv.gz"
JSON_FILE_2008

In [ ]:
df_2008 = pd.read_csv(JSON_FILE_2008)
df_2008.head()

In [ ]:
JSON_FILE_2009 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2009.csv.gz"
JSON_FILE_2009

In [ ]:
df_2009 = pd.read_csv(JSON_FILE_2009)
df_2009.head()

In [ ]:
JSON_FILE_2010 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2010.csv.gz"
JSON_FILE_2010

In [ ]:
df_2010 = pd.read_csv(JSON_FILE_2010)
df_2010.head()

In [ ]:
JSON_FILE_2011 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2011.csv.gz"
JSON_FILE_2011

In [ ]:
df_2011 = pd.read_csv(JSON_FILE_2011)
df_2011.head()

In [ ]:
JSON_FILE_2012 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2012.csv.gz"
JSON_FILE_2012

In [ ]:
df_2012 = pd.read_csv(JSON_FILE_2012)
df_2012.head()

In [ ]:
JSON_FILE_2013 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2013.csv.gz"
JSON_FILE_2013

In [ ]:
df_2013 = pd.read_csv(JSON_FILE_2013)
df_2013.head()

In [ ]:
JSON_FILE_2014 = f"C:/Users/fid24/OneDrive/Documents/Github/Project-3-/data/final_tmdb_data_2014.csv.gz"
JSON_FILE_2014

In [ ]:
df_2014 = pd.read_csv(JSON_FILE_2014)
df_2014.head()

In [110]:
df = pd.concat([df_2002, df_2003, df_2004, df_2005, df_2006, df_2008, df_2009, df_2010, df_2011, df_2012, df_2013, df_2014], ignore_index=True)
df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0096056,0.0,/95U3MUDXu4xSCmVLtWgargRipDi.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,109809.0,en,Crime and Punishment,...,0.0,126.0,"[{'english_name': 'Polish', 'iso_639_1': 'pl',...",Released,NaN,Crime and Punishment,0.0,5.500,11.0,NaN
2,tt0118926,0.0,/qR3Dk3ctnrrxkAI6I472RhamIbu.jpg,NaN,0.0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,20689.0,en,The Dancer Upstairs,...,5227348.0,132.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"An honest man caught in a world of intrigue, p...",The Dancer Upstairs,0.0,6.294,51.0,R
3,tt0119980,0.0,NaN,NaN,0.0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,563364.0,en,Random Shooting in LA,...,0.0,91.0,[],Released,NaN,Random Shooting in LA,0.0,0.000,0.0,NaN
4,tt0120679,0.0,/s04Ds4xbJU7DzeGVyamccH4LoxF.jpg,NaN,12000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",https://www.miramax.com/movie/frida,1360.0,en,Frida,...,56298474.0,123.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Prepare to be seduced.,Frida,0.0,7.455,1764.0,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44944,tt9558490,0.0,NaN,"{'id': 337019, 'name': '99 Cahaya Di Langit Er...",0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,337023.0,id,99 Cahaya di Langit Eropa: Part 2,...,0.0,99.0,"[{'english_name': 'German', 'iso_639_1': 'de',...",Released,NaN,99 Cahaya di Langit Eropa: Part 2,0.0,7.000,1.0,NaN
44945,tt9672244,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,665423.0,es,Le fils de Marie,...,0.0,81.0,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,NaN,Le fils de Marie,0.0,0.000,0.0,NaN
44946,tt9718528,0.0,NaN,NaN,20000.0,"[{'id': 18, 'name': 'Drama'}]",NaN,344109.0,pt,Dois Casamentos,...,0.0,75.0,"[{'english_name': 'Portuguese', 'iso_639_1': '...",Released,NaN,Dois Casamentos,0.0,5.000,3.0,NaN
44947,tt9853118,0.0,NaN,NaN,0.0,"[{'id': 80, 'name': 'Crime'}]",https://sakhamovie.ru/#!/post/59,594004.0,ru,Күрүөйэх,...,0.0,78.0,"[{'english_name': 'Russian', 'iso_639_1': 'ru'...",Released,NaN,Күрүөйэх,0.0,0.000,0.0,NaN


In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44949 entries, 0 to 44948
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                44949 non-null  object 
 1   adult                  44937 non-null  float64
 2   backdrop_path          20939 non-null  object 
 3   belongs_to_collection  2631 non-null   object 
 4   budget                 44937 non-null  float64
 5   genres                 44937 non-null  object 
 6   homepage               8185 non-null   object 
 7   id                     44937 non-null  float64
 8   original_language      44937 non-null  object 
 9   original_title         44937 non-null  object 
 10  overview               42097 non-null  object 
 11  popularity             44937 non-null  float64
 12  poster_path            37687 non-null  object 
 13  production_companies   44937 non-null  object 
 14  production_countries   44937 non-null  object 
 15  re

In [112]:
df.drop(columns='adult', inplace=True)
df.drop(columns='backdrop_path', inplace=True)
df.drop(columns='belongs_to_collection', inplace=True)
df.drop(columns='genres', inplace=True)
df.drop(columns='homepage', inplace=True)
df.drop(columns='id', inplace=True)
df.drop(columns='original_language', inplace=True)
df.drop(columns='overview', inplace=True)
df.drop(columns='popularity', inplace=True)
df.drop(columns='poster_path', inplace=True)
df.drop(columns='production_companies', inplace=True)
df.drop(columns='production_countries', inplace=True)
df.drop(columns='release_date', inplace=True)
df.drop(columns='runtime', inplace=True)
df.drop(columns='spoken_languages', inplace=True)
df.drop(columns='status', inplace=True)
df.drop(columns='tagline', inplace=True)
df.drop(columns='video', inplace=True)
df.drop(columns='vote_average', inplace=True)
df.drop(columns='vote_count', inplace=True)
df.drop(columns='original_title', inplace=True)
df

,imdb_id,budget,revenue,title,certification
0,0,NaN,NaN,NaN,NaN
1,tt0096056,0.0,0.0,Crime and Punishment,NaN
2,tt0118926,0.0,5227348.0,The Dancer Upstairs,R
3,tt0119980,0.0,0.0,Random Shooting in LA,NaN
4,tt0120679,12000000.0,56298474.0,Frida,R
...,...,...,...,...,...
44944,tt9558490,0.0,0.0,99 Cahaya di Langit Eropa: Part 2,NaN
44945,tt9672244,0.0,0.0,Le fils de Marie,NaN
44946,tt9718528,20000.0,0.0,Dois Casamentos,NaN
44947,tt9853118,0.0,0.0,Күрүөйэх,NaN


In [113]:
df.isna().sum()

imdb_id              0
budget              12
revenue             12
title               12
certification    36779
dtype: int64

In [114]:
df.dropna(subset=['budget'], inplace=True)
df.isna().sum()

imdb_id              0
budget               0
revenue              0
title                0
certification    36767
dtype: int64

In [115]:
df.duplicated().sum()

0

In [116]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44937 entries, 1 to 44948
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   imdb_id        44937 non-null  object 
 1   budget         44937 non-null  float64
 2   revenue        44937 non-null  float64
 3   title          44937 non-null  object 
 4   certification  8170 non-null   object 
dtypes: float64(2), object(3)
memory usage: 2.1+ MB


In [118]:
df.to_csv("Data/df_2002-2014_no_2007.csv.gz",compression='gzip',index=False)